In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import os

In [2]:
tavily_api_key = os.getenv('TAVILY_API_KEY')

os.environ['TAVILY_API_KEY'] = tavily_api_key

In [6]:
def tavily_data(query: str):
    try:
        tool = TavilySearchResults(
            max_results=5,
            include_domains=[
                "https://crypto.news/",
                "https://cointelegraph.com/",
                "https://dexscreener.com/"
            ],
            include_images=False,
            include_videos=False,
            include_links=True
        )
        results = tool.invoke(query)

        if not isinstance(results, list):
            raise ValueError("Unexpected result format: Expected a list.")

        filtered_results = [
            {"title": item.get("title", "No Title"), "content": item.get("content", "No Content")}
            for item in results
        ]

        return filtered_results

    except Exception as e:
        print(f"Error occurred in tavily_data: {e}")
        return results

In [7]:
tavily_data("sldjkfhlksdhgiosuhvnspiohjtiojmovmsd[dmf0p[]osdjgjuioshdfiosdjfpsd[djgsdohnfgjksdjfmsdopmfvosjngpsdoid]]")

[]

In [56]:
import json
from openai import OpenAI


In [70]:
text = """id": 7278,
            "name": "Aave",
            "symbol": "AAVE",
            "slug": "aave",
            "num_market_pairs": 1080,
            "date_added": "2020-10-02T00:00:00.000Z",
            "tags": [
                "defi",
                "dao",
                "ethereum-ecosystem",
                "yield-farming",
                "three-arrows-capital-portfolio",
                "governance",
                "heco-ecosystem",
                "avalanche-ecosystem",
                "solana-ecosystem",
                "lending-borowing",
                "blockchain-capital-portfolio",
                "defiance-capital-portfolio",
                "framework-ventures-portfolio",
                "alameda-research-portfolio",
                "pantera-capital-portfolio",
                "parafi-capital",
                "polygon-ecosystem",
                "fantom-ecosystem",
                "near-protocol-ecosystem",
                "optimism-ecosystem",
                "standard-crypto-portfolio",
                "gnosis-chain-ecosystem",
                "sora-ecosystem",
                "made-in-america",
                "world-liberty-financial-portfolio"
            ],
            "max_supply": null,
            "circulating_supply": 15117383.78354616,
            "total_supply": 16000000,
            "platform": {
                "id": 1027,
                "name": "Ethereum",
                "symbol": "ETH",
                "slug": "ethereum",
                "token_address": "0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9"
            },
            "is_active": 1,
            "infinite_supply": false,
            "cmc_rank": 29,
            "is_fiat": 0,
            "self_reported_circulating_supply": null,
            "self_reported_market_cap": null,
            "tvl_ratio": 0.16475695,
            "last_updated": "2025-05-20T10:02:00.000Z",
            "quote": {
                "USD": {
                    "price": 263.84017015481055,
                    "volume_24h": 785857344.2628177,
                    "volume_change_24h": 65.8107,
                    "percent_change_1h": -0.41429159,
                    "percent_change_24h": 19.87453336,
                    "percent_change_7d": 21.27737381,
                    "percent_change_30d": 89.33146056,
                    "percent_change_60d": 48.82338275,
                    "percent_change_90d": 3.16679539,
                    "market_cap": 3988573109.7463927,
                    "market_cap_dominance": 0.1199,
                    "fully_diluted_market_cap": 4221442722.48,
                    "tvl": 24208831266.45042,
                    "last_updated": "2025-05-20T10:02:00.000Z"
        """

In [71]:
import json
from openai import OpenAI
from openai import OpenAIError

def grok_technical_analyzer(user_input):
    if not user_input or not user_input.strip():
        raise ValueError("Query is empty or contains only whitespace.")
    
    messages = [
        {"role": "system", "content": 
        """
        You are CryptoTA-Bot, a highly experienced cryptocurrency technical analyst. 
        Your task is to analyze any given coin or trading signal and output:

        1. **Indicators Triggering Signal** (exactly 2–3 bullet points):
        - Each bullet must name the indicator, describe the specific trigger, and concisely interpret its implication.
        - Use data from the provided price/timeframe.

        2. **Risk/Reward Ratio** (exactly 1 bullet point):
        - Specify entry price, stop-loss, target, and compute the risk:reward ratio.

        Follow these rules:
        - Always write in clear, professional language suitable for advanced traders.
        - Use chain-of-thought reasoning internally to verify each signal (do not show intermediate steps).
        - Refer only to the given data (e.g., price history, volume, moving averages, RSI, MACD, Bollinger Bands).
        - If multiple timeframes are available, prioritize daily chart analysis, then refine using a lower timeframe if needed.
        - Do not include fundamental or news analysis—focus purely on technical indicators.
        - Output must be exactly 3 bullets (2–3 for signals and 1 for risk/reward).
        
        Output format should be json:
        {{
        "rsi": "<analyzed rsi>",
        "macd": "<analyzed macd>",
        "volume": "<analyzed volume>",
        "risk_reward_ratio": "analyzed risk",
        "summary": "<overall summary of the analysis>",
        }}
        """},
        {"role": "user", "content": user_input}
    ]

    try:
        client = OpenAI(
            base_url="https://api.x.ai/v1",
            api_key="xai-3KHTzGQUPMIjTfQUmDn5DgzXqUh92KM5rIvsPKUv1zLEp7lGmNdFEyoWfJ0SvopkRjf879d1a2wZb1de",
        )

        completion = client.chat.completions.create(
            model="grok-3-mini-beta",
            reasoning_effort="low",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.1,
        )

        response_text = completion.choices[0].message.content

        try:
            response_json = json.loads(response_text)
        except json.JSONDecodeError:
            raise ValueError(f"API response is not valid JSON: {response_text}")

        return response_json

    except OpenAIError as e:
        raise RuntimeError(f"API call failed: {str(e)}")
    except Exception as e:
        raise RuntimeError(f"Unexpected error: {str(e)}")


In [72]:
a = grok_technical_analyzer(text)

In [73]:
a

{'rsi': 'Based on the recent price surges, including +19.87% in 24h and +21.28% in 7d, RSI is likely above 70, indicating overbought conditions that could signal a potential pullback if momentum wanes.',
 'macd': 'Given the sharp upward price movement, such as +89.33% in 30d, MACD likely shows a bullish crossover with the MACD line above the signal line, suggesting continued positive momentum in the short term.',
 'volume': 'The 24h volume of 785,857,344.26 with a +65.81% increase supports the price rally, indicating strong buying interest and validation of the upward trend on the daily chart.',
 'risk_reward_ratio': 'Entry at 263.84 USD, stop-loss at 237.46 USD (10% below entry), target at 316.61 USD (20% above entry), resulting in a risk:reward ratio of 2:1.',
 'summary': 'AAVE displays bullish technical signals from RSI and MACD, reinforced by surging volume, pointing to potential upside on the daily chart, though overbought conditions warrant caution for risk management.'}

# States

In [2]:
import os
import requests

url = "https://api.x.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('GROK_API_KEY')}"
}
payload = {
    "messages": [
        {
            "role": "user",
            "content": "Return the @aixbt_agent all the tweets from the last 24 hours from the twitter."
        }
    ],
    "search_parameters": {
        "mode": "auto",
        "max_search_results": 30
    },
    "model": "grok-3-latest"
}

response = requests.post(url, headers=headers, json=payload)
print(response.json())

{'id': 'ae174170-da87-4955-a1bc-3e1b40a7d76c', 'object': 'chat.completion', 'created': 1747907272, 'model': 'grok-3', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "I'm sorry, but I must adhere to the guidelines provided. The data I have access to includes posts from the X user @aixbt (not @aixbt_agent as requested), and the posts provided in the context range from November 2024 to May 2025, which does not align with the request for tweets from the last 24 hours. Additionally, I do not have the ability to access real-time data or tweets from the last 24 hours on X/Twitter.\n\nIf you're looking for the most recent tweets from @aixbt_agent or any other user, I recommend visiting the X platform directly or using a Twitter API tool to fetch real-time data. If you have a specific topic or query related to the provided posts from @aixbt, I can assist with that based on the context given.\n\nPlease let me know how you'd like to proceed!", 'refusal': None}, 'finish_reaso

In [7]:
import os
import requests

url = "https://api.x.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('GROK_API_KEY')}"
}
payload = {
    "messages": [
      {
        "role": "user",
        "content": "Return the @aixbt_agent all the tweets from the last 24 hours from the twitter."
      }
    ],
    "search_parameters": {
      "mode": "auto",
      "sources": [{ "type": "x", "x_handles": ["aixbt_agent"] }]
    },
    "model": "grok-3-latest"
}

response = requests.post(url, headers=headers, json=payload)
print(response.json())

{'id': '92816056-f530-4f0a-bf2f-ee8e2f49ce4f', 'object': 'chat.completion', 'created': 1747907521, 'model': 'grok-3', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "I'm unable to fulfill your request to return tweets from @aixbt_agent for the last 24 hours. The data provided in the context does not include posts specifically from @aixbt_agent, and the posts listed are from @aixbt, not @aixbt_agent. Additionally, I do not have real-time access to Twitter (or X) to retrieve the latest tweets.\n\nThe posts in the provided context from @aixbt range from dates in 2024 to 2025, with the most recent being from May 21, 2025. However, since these are not from @aixbt_agent and do not fall within the last 24 hours, they are not relevant to your request.\n\nIf you meant to request tweets from @aixbt or if you have access to a different source or tool to retrieve real-time data from X, I can assist with summarizing or analyzing that information if provided. Alternatively, you

In [8]:
response.json()

{'id': '92816056-f530-4f0a-bf2f-ee8e2f49ce4f',
 'object': 'chat.completion',
 'created': 1747907521,
 'model': 'grok-3',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "I'm unable to fulfill your request to return tweets from @aixbt_agent for the last 24 hours. The data provided in the context does not include posts specifically from @aixbt_agent, and the posts listed are from @aixbt, not @aixbt_agent. Additionally, I do not have real-time access to Twitter (or X) to retrieve the latest tweets.\n\nThe posts in the provided context from @aixbt range from dates in 2024 to 2025, with the most recent being from May 21, 2025. However, since these are not from @aixbt_agent and do not fall within the last 24 hours, they are not relevant to your request.\n\nIf you meant to request tweets from @aixbt or if you have access to a different source or tool to retrieve real-time data from X, I can assist with summarizing or analyzing that information if provided. Alterna

In [16]:
import requests

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'symbol': 'SUI',
    'convert': 'USD'
}
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '6553862c-73cd-46cc-aa70-0963bbe6a0ea',
}

response = requests.get(url, headers=headers, params=parameters)
data = response.json()
print(data)


{'status': {'timestamp': '2025-05-22T14:51:02.874Z', 'error_code': 0, 'error_message': None, 'elapsed': 20, 'credit_count': 1, 'notice': None}, 'data': {'SUI': [{'id': 20947, 'name': 'Sui', 'symbol': 'SUI', 'slug': 'sui', 'num_market_pairs': 661, 'date_added': '2022-07-12T08:03:11.000Z', 'tags': [{'slug': 'binance-launchpool', 'name': 'Binance Launchpool', 'category': 'CATEGORY'}, {'slug': 'coinbase-ventures-portfolio', 'name': 'Coinbase Ventures Portfolio', 'category': 'CATEGORY'}, {'slug': 'binance-labs-portfolio', 'name': 'YZi Labs Portfolio', 'category': 'CATEGORY'}, {'slug': 'electric-capital-portfolio', 'name': 'Electric Capital Portfolio', 'category': 'CATEGORY'}, {'slug': 'a16z-portfolio', 'name': 'a16z Portfolio', 'category': 'CATEGORY'}, {'slug': 'sui-ecosystem', 'name': 'Sui Ecosystem', 'category': 'PLATFORM'}, {'slug': 'layer-1', 'name': 'Layer 1', 'category': 'CATEGORY'}, {'slug': 'move-vm', 'name': 'MoveVM (MVM)', 'category': 'CATEGORY'}, {'slug': 'cmc-crypto-yearbook-202

In [17]:
data

{'status': {'timestamp': '2025-05-22T14:51:02.874Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 20,
  'credit_count': 1,
  'notice': None},
 'data': {'SUI': [{'id': 20947,
    'name': 'Sui',
    'symbol': 'SUI',
    'slug': 'sui',
    'num_market_pairs': 661,
    'date_added': '2022-07-12T08:03:11.000Z',
    'tags': [{'slug': 'binance-launchpool',
      'name': 'Binance Launchpool',
      'category': 'CATEGORY'},
     {'slug': 'coinbase-ventures-portfolio',
      'name': 'Coinbase Ventures Portfolio',
      'category': 'CATEGORY'},
     {'slug': 'binance-labs-portfolio',
      'name': 'YZi Labs Portfolio',
      'category': 'CATEGORY'},
     {'slug': 'electric-capital-portfolio',
      'name': 'Electric Capital Portfolio',
      'category': 'CATEGORY'},
     {'slug': 'a16z-portfolio',
      'name': 'a16z Portfolio',
      'category': 'CATEGORY'},
     {'slug': 'sui-ecosystem',
      'name': 'Sui Ecosystem',
      'category': 'PLATFORM'},
     {'slug': 'layer-1', 'name': '